In [1]:
import autokeras as ak
from autokeras.nn.generator import MobileNetV2Generator,CnnGenerator, ResNetGenerator
from copy import deepcopy

model = MobileNetV2Generator(10, (32, 32, 3)).generate(10,32).produce_model()
model.set_weight_to_graph()
graph = model.graph

print(model)


TorchModel(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
  (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
  (12): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
  (15): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [2]:
weighted_layer_ids = graph.deep_layer_ids()

print(weighted_layer_ids)


[10, 18, 26]


In [3]:
from autokeras.net_transformer import to_deeper_graph

temp_graph = to_deeper_graph(deepcopy(graph))


In [4]:
#temp_graph.weighted = False
temp_model = temp_graph.produce_model()
print(temp_model)

TorchModel(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
  (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
  (12): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
  (15): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [5]:
from autokeras.nn.layers import is_layer, LayerType

for i in range(len(temp_graph.layer_list)):
    layer=temp_graph.layer_list[i]    
    if is_layer(layer, LayerType.CONV):
        if temp_graph.weighted:
            teacher_w, teacher_b = layer.get_weights()
            teacher_w = teacher_w.shape
            teacher_b = teacher_b.shape
        else:
            teacher_w = 0
            teacher_b = 0
        print("layer_id:{}, input_channel:{}, filters:{}, groups:{}, teacher_w:{}, teacher_b: {}".format(i,layer.input_channel,layer.filters,layer.groups,teacher_w, teacher_b))
    elif is_layer(layer, LayerType.DENSE):
        if temp_graph.weighted:
            teacher_w, teacher_b = layer.get_weights()
            teacher_w = teacher_w.shape
            teacher_b = teacher_b.shape
        else:
            teacher_w = 0
            teacher_b = 0
        print("layer_id:{}, input_shape:{},teacher_w:{}, teacher_b: {}".format(i,layer.input.shape,teacher_w, teacher_b))

layer_id:0, input_channel:3, filters:32, groups:1, teacher_w:(32, 3, 3, 3), teacher_b: (32,)
layer_id:3, input_channel:32, filters:32, groups:1, teacher_w:(32, 32, 1, 1), teacher_b: (32,)
layer_id:6, input_channel:32, filters:32, groups:32, teacher_w:(32, 1, 3, 3), teacher_b: (32,)
layer_id:9, input_channel:32, filters:16, groups:1, teacher_w:(16, 32, 1, 1), teacher_b: (16,)
layer_id:11, input_channel:16, filters:96, groups:1, teacher_w:(96, 16, 1, 1), teacher_b: (96,)
layer_id:14, input_channel:96, filters:96, groups:96, teacher_w:(96, 1, 3, 3), teacher_b: (96,)
layer_id:17, input_channel:96, filters:24, groups:1, teacher_w:(24, 96, 1, 1), teacher_b: (24,)
layer_id:19, input_channel:24, filters:144, groups:1, teacher_w:(144, 24, 1, 1), teacher_b: (144,)
layer_id:22, input_channel:144, filters:144, groups:144, teacher_w:(144, 1, 3, 3), teacher_b: (144,)
layer_id:25, input_channel:144, filters:24, groups:1, teacher_w:(24, 144, 1, 1), teacher_b: (24,)
layer_id:27, input_channel:24, filte

In [6]:
import torch

x = torch.randn(2,3,32,32)
y2 = temp_model(x)
print(y2.size())

torch.Size([2, 10])


In [7]:
import os
from graphviz import Digraph

from autokeras.utils import pickle_from_file


def to_pdf(graph, path):
    dot = Digraph(comment='The Round Table')

    for index, node in enumerate(graph.node_list):
        dot.node(str(index), str(node.shape))

    for u in range(graph.n_nodes):
        for v, layer_id in graph.adj_list[u]:
            dot.edge(str(u), str(v), str(graph.layer_list[layer_id]))

    dot.render(path)


In [8]:
to_pdf(temp_graph, 'deeper')

In [9]:
print(temp_graph.adj_list)

{0: [(1, 0)], 1: [(2, 1)], 2: [(3, 2)], 3: [(4, 3)], 4: [(5, 4)], 5: [(6, 5)], 6: [(7, 6)], 7: [(8, 7)], 8: [(9, 8)], 9: [(10, 9)], 10: [(11, 10)], 11: [(12, 11)], 12: [(13, 12)], 13: [(14, 13)], 14: [(15, 14)], 15: [(16, 15)], 16: [(17, 16)], 17: [(18, 17)], 18: [(33, 18)], 19: [(20, 19)], 20: [(21, 20)], 21: [(22, 21)], 22: [(23, 22)], 23: [(24, 23)], 24: [(25, 24)], 25: [(26, 25)], 26: [(27, 26)], 27: [(28, 27)], 28: [(29, 28)], 29: [(30, 29)], 30: [(31, 30)], 31: [(32, 31)], 32: [], 33: [(34, 32)], 34: [(35, 33)], 35: [(36, 34)], 36: [(37, 35)], 37: [(38, 36)], 38: [(39, 37)], 39: [(40, 38)], 40: [(19, 39)]}


In [10]:
print(temp_graph.deep_layer_ids())

[10, 18, 39, 26]


In [11]:
print(graph.adj_list)

{0: [(1, 0)], 1: [(2, 1)], 2: [(3, 2)], 3: [(4, 3)], 4: [(5, 4)], 5: [(6, 5)], 6: [(7, 6)], 7: [(8, 7)], 8: [(9, 8)], 9: [(10, 9)], 10: [(11, 10)], 11: [(12, 11)], 12: [(13, 12)], 13: [(14, 13)], 14: [(15, 14)], 15: [(16, 15)], 16: [(17, 16)], 17: [(18, 17)], 18: [(19, 18)], 19: [(20, 19)], 20: [(21, 20)], 21: [(22, 21)], 22: [(23, 22)], 23: [(24, 23)], 24: [(25, 24)], 25: [(26, 25)], 26: [(27, 26)], 27: [(28, 27)], 28: [(29, 28)], 29: [(30, 29)], 30: [(31, 30)], 31: [(32, 31)], 32: []}


In [12]:
pre_layer = None
for layer_id in temp_graph.get_main_chain_layers():
    layer = temp_graph.layer_list[layer_id]
    #if is_layer(layer, LayerType.GLOBAL_POOL):
    #    break
    if is_layer(layer, LayerType.ADD) or is_layer(layer, LayerType.CONCAT):
        continue
    #if is_layer(layer, LayerType.CONV) and (pre_layer is not None) and is_layer(pre_layer, LayerType.BATCH_NORM):
    print(layer_id)
    pre_layer = layer


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
32
33
34
35
36
37
38
39
19
20
21
22
23
24
25
26
27
28
29
30
31
